<a href="https://colab.research.google.com/github/agatagruza/private-ai/blob/master/SPAIC_Project16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 16: Encrypted Deep Learning in Keras

You will provifr private prediction by:</br>
**Step 1**: Train your model with normal Keras.</br>
**Step 2**: Secure and serve your machine learning model (server).</br>
**Step 3**: Query the secured model to receive private predictions (client). </br>

*Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials*

##Step1: Train Your Model in Keras
For more info about Keras, check [Keras](https://keras.io/) and  [Keras documentation](https://keras.io/)

To train a classification  model with normal Keras use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Below example is borrowed from the [reference Keras repository](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py). 

In [1]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11493376/11490434 [==============================] - 0s 0us/step


W0729 02:58:03.774518 139912994658176 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 95us/sample - loss: 2.2995 - acc: 0.1456 - val_loss: 2.2985 - val_acc: 0.1639
Epoch 2/2
60000/60000 [==============================] - 2s 32us/sample - loss: 2.2981 - acc: 0.1754 - val_loss: 2.2970 - val_acc: 0.1855
Test loss: 2.2970151245117187
Test accuracy: 0.1855


In [0]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

##Step 2: Load and Serve the Model
To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).</br>

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run hook = sy.KerasHook(tf.keras). This will add three important new methods to the Keras Sequential class: </br>

*   ***share:*** will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
*   ***serve:*** this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
*   ***shutdown_workers***: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker. </br>

To earn more about MPC, go to [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

W0729 02:58:39.404854 139912994658176 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0729 02:58:39.506308 139912994658176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



##Model
Below model is almost the saem as before, except it includes a batch_input_shape. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [0]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

## Load Pre-trained Weights

In [0]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

##Step 3: Setup Your Worker Connectors

Now we connect to the TFEWorkers (alice, bob, and carol) required by TF Encrypted to perform private predictions. For each TFEWorker, we have to specify a host.

These workers run a TensorFlow server, which we can either manage manually (AUTO = False) or ask the workers to manage for you (AUTO = True). If choosing to manually manage them, we will be instructed to execute a terminal command on each worker's host device after calling model.share() below. If all workers are hosted on a single device (e.g. localhost), we can choose to have Syft automatically manage the worker's TensorFlow server.

In [0]:
AUTO = False

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

##Step 4: Split the Model Into Shares

In [0]:
model.share(alice, bob, carol)

##Step 5: Launch 3 Servers

From other terminal we need to launch:</br>
***python -m tf_encrypted.player --config /tmp/tfe.config server0</br>
python -m tf_encrypted.player --config /tmp/tfe.config server1</br>
python -m tf_encrypted.player --config /tmp/tfe.config server2***</br> 

##Step 6: Serve the Model

In [0]:
model.serve(num_requests=3)

##Step 7: Run the Client

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import syft as sy

In [0]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
num_classes = 10
input_shape = (1, 28, 28, 1)
output_shape = (1, num_classes)

In [17]:
client = sy.TFEWorker()

alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')

client.connect_to_model(input_shape, output_shape, alice, bob, carol)

W0729 03:26:04.774584 139912994658176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/tensor/native.py:403: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0729 03:26:04.785006 139912994658176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/config.py:300: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0729 03:26:04.785733 139912994658176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/config.py:87: The name tf.GraphOptions is deprecated. Please use tf.compat.v1.GraphOptions instead.

I0729 03:26:04.788068 139912994658176 session.py:55] Starting session on target 'grpc://localhost:4000' using config graph_options {
}



In [0]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [0]:
for image, expected_label in zip(images, expected_labels):

    res = client.query_model(image.reshape(1, 28, 28, 1))
    predicted_label = np.argmax(res)

    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "wrongly",
        predicted_label))

##Shutdown the Servers

Once above request hit the limit above, the model will no longer be available for serving requests. However it's still secret shared between the three workers above. Therefore we should kill the workers.

In [0]:
model.shutdown_workers()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config /tmp/tfe.config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))